# RRT: basic finding example
* Path finding is using sample-based (RRT) methods rather than constrained optimization to find a collision free path
* Path finding is much easier if a final robot pose is given. We here use IK (formulated as KOMO problem) to first compute a final configuration 'qT'. (Path optimization does this jointly with optimizing the path.)
* Then we can pass the current state and qT to a bi-directional RRT to find a collision free path.
* Note that BotOp (the interface to sim/real) is opened only after we computed the motion. We simply pass the motion to be played by the sim/real.

In [1]:
import robotic as ry
import time

first a minimalistic example for testing:

In [2]:

C = ry.Config()
C.addFrame("base") .setPosition([0,0,.5])

C.addFrame("ego", "base") \
    .setJoint(ry.JT.transXYPhi, [-1.,1.,-1.,1.,-3.,3.]) \
    .setRelativePosition([.2, .0, .0]) \
    .setShape(ry.ST.ssBox, size=[.05, .3, .05, .01]) \
    .setColor([0, 1., 1.]) \
    .setContact(1)

C.addFrame("obstacle") \
    .setPosition([.0, .0, .5]) \
    .setShape(ry.ST.ssBox, size=[.05, .3, .05, .01]) \
    .setColor([1, .5, 0]) \
    .setContact(1)

C.view(False)

0

In [3]:
q0 = [-.2, 0, 0]
qT = [+.2, 0, 0]

ry.params_clear()
ry.params_add({'rrt/stepsize':.1, 'rrt/verbose': 3}) #this makes it very slow, and displays result, and wait keypress..

rrt = ry.PathFinder()
rrt.setProblem(C, [q0], [qT])
ret = rrt.solve()
print(ret)
path = ret.x

ry.params_print()

{ time: 0.339952, evals: 174, done: 1, feasible: 1, sos: -1, f: -1, ineq: -1, eq: -1 }
-- kin.cpp:fcl:2078(0)   SKIPPING from FCL interface: base
-- kin.cpp:fcl:2076(0)   adding to FCL interface: ego
-- kin.cpp:fcl:2076(0)   adding to FCL interface: obstacle
RRT queries=436 tree sizes = 41 39
  -- rrt success: queries:762 tree sizes: 69 70
  path-length=31
rrt/stepsize: 0.1,
rrt/verbose: 3,
rrt/subsamples: 4,
seed: 0

In [4]:
del rrt

print('path length:', path.shape)
# display the path
for t in path:
    C.setJointState(t)
    C.view()
    time.sleep(1./path.shape[0])

path length: (31, 3)


In [5]:
#this prints all parameters used by the rrt:
ry.params_print()

rrt/stepsize: 0.1,
rrt/verbose: 3,
rrt/subsamples: 4,
seed: 0

In [6]:
C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandasTable.g'))
C.view()

0

In [7]:
C.addFrame('boxR','table') \
    .setRelativePosition([.15,0,.1]) \
    .setShape(ry.ST.ssBox, size=[.1,.1,.1,.02]) \
    .setColor([1,1,0])
C.addFrame('boxL','table') \
    .setRelativePosition([-.15,0,.1]) \
    .setShape(ry.ST.ssBox, size=[.1,.1,.1,.02]) \
    .setColor([1,.5,0])
C.view()

0

In [8]:
# store the start configuration
q0 = C.getJointState()

In [9]:
# compute a goal configuration
komo = ry.KOMO()
komo.setConfig(C, True)
komo.setTiming(1., 1, 5., 0)
komo.addControlObjective([], 0, 1e-0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq);
komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq);
komo.addObjective([], ry.FS.positionDiff, ['r_gripper', 'boxL'], ry.OT.eq, [1e1]);
komo.addObjective([], ry.FS.positionDiff, ['l_gripper', 'boxR'], ry.OT.eq, [1e1]);

In [10]:
ret = ry.NLP_Solver() \
    .setProblem(komo.nlp()) \
    .setOptions( stopTolerance=1e-2, verbose=4 ) \
    .solve()
print(ret)

{ time: 0.14005, evals: 181, done: 1, feasible: 1, sos: 4.60525, f: 0, ineq: 0, eq: 0.00832905 }====nlp==== method:AugmentedLagrangian bounded: yes

==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
----newton---- initial point f(x):118.878 alpha:1 beta:1
--newton-- it:   1  |Delta|:        0.2  alpha:          1  evals:   2  f(y):    61.1067  ACCEPT
--newton-- it:   2  |Delta|:        0.2  alpha:          1  evals:   3  f(y):    25.6233  ACCEPT
--newton-- it:   3  |Delta|:        0.2  alpha:          1  evals:   4  f(y):     13.394  ACCEPT
--newton-- it:   4  |Delta|:        0.2  alpha:          1  evals:   5  f(y):    8.78883  ACCEPT
--newton-- it:   5  |Delta|:        0.2  alpha:          1  evals:   6  f(y):    5.72548  ACCEPT
--newton-- it:   6  |Delta|:   0.199662  alpha:          1  evals:   7  f(y):     4.0143  ACCEPT
--newton-- it:   7  |Delta|:        0.2  alpha:          1  evals:   8  f(y):    3.69354  ACCEPT
--newton-- it:   8  |Delta|:   0.124542  alpha:          1  evals:   9  f(y

In [11]:
# that's the goal configuration
qT = komo.getPath()[0]
C.setJointState(qT)
C.view(False, "IK solution")

0.0335182  alpha:          1  evals:  11  f(y):    3.52013  ACCEPT
--newton-- it:  11  |Delta|:  0.0219181  alpha:          1  evals:  12  f(y):    3.51622  ACCEPT
--newton-- stopping: 'absMax(Delta)<options.stopTolerance'
==nlp== it:   0  evals:  12  A(x):    3.51622  f:     3.1564  g:          0  h:    1.44686  |x-x'|:    1.01145 	stop:DeltaConverge
==nlp== it:   1  evals:  12  A(x):    5.67514  mu:5
--newton-- it:  12  |Delta|:   0.189886  alpha:          1  evals:  13  f(y):    4.21808  ACCEPT
--newton-- it:  13  |Delta|:  0.0892271  alpha:          1  evals:  14  f(y):     4.2193  reject (lineSearch:0)
                    (line search)        alpha:        0.5  evals:  15  f(y):    4.18509  ACCEPT
--newton-- it:  14  |Delta|:  0.0511295  alpha:       0.75  evals:  16  f(y):    4.17105  ACCEPT
--newton-- it:  15  |Delta|:  0.0602826  alpha:          1  evals:  17  f(y):     4.2128  reject (lineSearch:0)
                    (line search)        alpha:        0.5  evals:  18  f(y):  

0

In [12]:
#define a path finding problem
rrt = ry.PathFinder()
rrt.setProblem(C, [q0], [qT])

-- kin.cpp:fcl:2078(0)   SKIPPING from FCL interface: world
-- kin.cpp:fcl:2078(0)   SKIPPING from FCL interface: table_base
-- kin.cpp:fcl:2076(0)   adding to FCL interface: table
-- kin.cpp:fcl:2078(0)   SKIPPING from FCL interface: l_panda_base
-- kin.cpp:fcl:2078(0)   SKIPPING from FCL interface: l_panda_link0
-- kin.cpp:fcl:2078(0)   SKIPPING from FCL interface: l_panda_link0_0
-- kin.cpp:fcl:2078(0)   SKIPPING from FCL interface: l_panda_joint1_origin
-- kin.cpp:fcl:2078(0)   SKIPPING from FCL interface: l_panda_joint1
-- kin.cpp:fcl:2078(0)   SKIPPING from FCL interface: l_panda_link1
-- kin.cpp:fcl:2078(0)   SKIPPING from FCL interface: l_panda_link1_0
-- kin.cpp:fcl:2078(0)   SKIPPING from FCL interface: l_panda_joint2_origin
-- kin.cpp:fcl:2078(0)   SKIPPING from FCL interface: l_panda_joint2
-- kin.cpp:fcl:2078(0)   SKIPPING from FCL interface: l_panda_link2
-- kin.cpp:fcl:2078(0)   SKIPPING from FCL interface: l_panda_link2_0
-- kin.cpp:fcl:2078(0)   SKIPPING from FCL inter

In [13]:
ret = rrt.solve()
print(ret)
path = ret.x

RRT queries=502 tree sizes = 70 32
{ time: 0.530418, evals: 177, done: 1, feasible: 1, sos: -1, f: -1, ineq: -1, eq: -1 }
  -- rrt success: queries:1028 tree sizes: 120 106
  path-length=37


In [14]:
# display the path
for t in range(0, path.shape[0]-1):
    C.setJointState(path[t])
    C.view()
    time.sleep(.1)

In [15]:
# run the path with botop
C.setJointState(q0)
ry.params_add({'botsim/verbose': 1., 'physx/motorKp': 10000., 'physx/motorKd': 1000.})
bot = ry.BotOp(C, False)
bot.home(C)

In [16]:
bot.moveAutoTimed(path, 1., 1.)
while bot.getTimeToEnd()>0:
    bot.sync(C, .1)

In [17]:
del bot

-- bot.cpp:~BotOp:128(0) shutting down BotOp...
-- simulation.cpp:~BotThreadedSim:56(0) shutting down SimThread
-- simulation.cpp:~Simulation:149(0) shutting down Simulation


In [18]:
del rrt
del C